In [1]:
import torch
import numpy as np
from utils.generate_csbm import *
from utils.train import *
from utils.utils import *
from models.models import *
from models.setup import *
import networkx as nx
import pickle

In [2]:
csbm = cSBM(N=200, p=100, d=5, mu=0.5, l=2)
csbm.generate_node_parameters()
csbm.phi

0.8886112459769363

In [11]:
csbm.generate_node_data(method="SNC", n_local=40)

In [6]:
ids = np.arange(200)
np.random.shuffle(ids)
train_ids = ids[0:20]
csbm.v[train_ids].sum()

0

In [7]:
train_ids
val_ids = ids[20:40]
test_ids = ids[40:]

In [62]:
def exp_csbm(ith, method, folder_path, csbm, train_ids, val_ids, test_ids, 
             num_communication=100, batch_size=1,
             learning_rate=0.01, I=10, gradient=True, noise=False,
             Print=False, print_time=1,
             resume=False,
             bias=False,
             save=True):
    

    torch.manual_seed(0)
    init_mlp = MLP(csbm.Xs[0].shape[1], 64, 2, bias)
        
    if (gradient == False):
        grad = "no_grad/"
    elif (gradient == True and noise==False):
        grad = "biased_grad/"
    elif (gradient == True and noise==True):
        grad = "noisy_grad/"
      
    A_tilde = calculate_Atilde(csbm.A, K=10, alpha=0.9)
    server = set_up_NC(csbm.Xs, csbm.ys, init_mlp, A_tilde, train_ids, val_ids, test_ids)
   
        
    tl, ta, vl, va = train_NC(server, num_communication, batch_size, learning_rate, I,
                              gradient, noise, 
                              Print, print_time)
    if save:    
        np.save(folder_path + method + grad + "I" + str(I) + "/tl_" + str(ith), tl)
        np.save(folder_path + method + grad + "I" + str(I) + "/ta_" + str(ith), ta)
        np.save(folder_path + method + grad + "I" + str(I) + "/vl_" + str(ith), vl)
        np.save(folder_path + method + grad + "I" + str(I) + "/va_" + str(ith), va)

        PATH = folder_path + method + grad + "I" + str(I) + "/model_" + str(ith)
        torch.save({
                'best_model_state_dict': server.best_cmodel.state_dict(),
                'learning_rate': learning_rate,
                'test_acc': server.eval_test()[1],
                'model_state_dict': server.cmodel.state_dict(),
                'best_valloss': server.best_valloss,
                'best_valacc': server.best_valacc,
                }, PATH)

In [15]:
init_mlp = MLP(csbm.Xs.shape[2], 64, 2, bias=False)

In [16]:
A_tilde = calculate_Atilde(csbm.A, K=10, alpha=0.9)
server = set_up_NC(csbm.Xs, csbm.ys, init_mlp, A_tilde, train_ids, val_ids, test_ids)

In [17]:
tl, ta, vl, va = train_NC(server, num_communication=500, batch_size=40, learning_rate=0.1, I=10,
                          Print=True, print_time=10)

Communication: 1 Average train loss: 0.69329 Average train accuracy: 0.500 Average val loss: 0.69419 Average val accuracy: 0.450
Communication: 11 Average train loss: 0.69261 Average train accuracy: 0.500 Average val loss: 0.69337 Average val accuracy: 0.450
Communication: 21 Average train loss: 0.69191 Average train accuracy: 0.504 Average val loss: 0.69256 Average val accuracy: 0.450
Communication: 31 Average train loss: 0.69117 Average train accuracy: 0.615 Average val loss: 0.69174 Average val accuracy: 0.450
Communication: 41 Average train loss: 0.69034 Average train accuracy: 0.829 Average val loss: 0.69086 Average val accuracy: 1.000
Communication: 51 Average train loss: 0.68937 Average train accuracy: 0.894 Average val loss: 0.68987 Average val accuracy: 1.000
Communication: 61 Average train loss: 0.68822 Average train accuracy: 0.876 Average val loss: 0.68873 Average val accuracy: 0.950
Communication: 71 Average train loss: 0.68682 Average train accuracy: 0.849 Average val los

In [ ]:
server.eval_test()